In [1]:
from openai import OpenAI

client = OpenAI()

In [3]:
from ipywebrtc import CameraStream, AudioRecorder

camera = CameraStream(constraints={"audio": True, "video": False})

recorder = AudioRecorder(stream=camera)

recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [4]:
with open('recording.webm', 'wb') as f:
  print(recorder.audio.value)
  f.write(recorder.audio.value)

In [5]:
audio_file = open("customer_recording.wav", "rb")

transcription = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)

print(transcription.text)


Thank you.


In [7]:
response = client.moderations.create(input=transcription.text)

output = response.results[0]

output.to_dict()

{'categories': {'harassment': False,
  'harassment/threatening': False,
  'hate': False,
  'hate/threatening': False,
  'self-harm': False,
  'self-harm/instructions': False,
  'self-harm/intent': False,
  'sexual': False,
  'sexual/minors': False,
  'violence': False,
  'violence/graphic': False},
 'category_scores': {'harassment': 1.7664155166130513e-05,
  'harassment/threatening': 1.7991452523347107e-06,
  'hate': 1.4257331031330978e-06,
  'hate/threatening': 1.8216221420175316e-08,
  'self-harm': 5.998155302222585e-06,
  'self-harm/instructions': 2.5065692170755938e-06,
  'self-harm/intent': 1.2040726687700953e-05,
  'sexual': 0.0005009017768315971,
  'sexual/minors': 3.055642082472332e-06,
  'violence': 2.020822648773901e-05,
  'violence/graphic': 2.0404590031830594e-05},
 'flagged': False}

In [18]:
from sklearn.metrics.pairwise import cosine_similarity


def get_embeddings(text):
  return client.embeddings.create(
    model="text-embedding-ada-002",
    input=text,
    encoding_format="float"
  ).data[0].embedding


# 임베딩을 만들어봅시다!
no_mon_embed      = get_embeddings("돈이 없다")
timetravel_embed  = get_embeddings("이혼")
bodysnat_embed    = get_embeddings("불륜")
reincarn_embed    = get_embeddings("재산분할")

# 미니 VectorDB 를 만들어봅시다..
solutions ={
    "돈이 없다": {
        "embeddings": no_mon_embed,
        "solutions": ["할인 이벤트가 있는지 확인한다!"]
    },
    "이혼": {
        "embeddings": timetravel_embed,
        "solutions": ["배우자와 결별을 한다"]
    },
    "불륜": {
        "embeddings": bodysnat_embed,
        "solutions": ["배우자의 불륜 또는 본인의 불륜으로인해 이혼을 한다."]
    },
    "재산분할": {
        "embeddings": reincarn_embed,
        "solutions": ["더 좋은 조건으로 재산분할을 한다."]
    }
}

def get_highest_match(problem_embed):
    # 사용자의 문제를 받고, 우리의 미니 DB 에서 제일 가까운 솔루션을 찾아낼 것입니다!
    results = []
    for key, value in solutions.items():
        sim = cosine_similarity([problem_embed], [value['embeddings']])[0][0]
        results.append((key, sim, value['solutions']))

    # 유사도로 정리
    results.sort(key=lambda x: x[1], reverse=True)

    # 제일 유사한 하나만 가져오기
    return results[0]

def get_solution(problem) -> str:
    # 사용자의 문제를 임베딩으로 바꿈
    problem_embed = get_embeddings(problem)

    # 제일 가까운 카테고리의 솔루션을 가지고 옴
    solution_found = get_highest_match(problem_embed)[2][0]
    return solution_found

def get_problem(user_query):
  # 사용자의 질문을 듣고 카테고리를 선별함
  return client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {"role": "system", "content": """사용자의 질문을 듣고 그 문제를 해결할 수 있는 방법을 다음 다섯가지 중에서 선택해줘:
            '돈이 없다', '이혼', '불륜', '재산분할'
            이 중에 선택할 옵션이 없으면 그냥 아무 대답도 하지 마.
            """
            },
            {"role": "user", "content": user_query},
        ]
    ).choices[0].message.content

def get_detailed_solution(user_query):
  # 모든 자료가 모였으니 이제 마지막 답변을 생성함
  solution = get_solution(get_problem(user_query))
  return client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {"role": "system", "content": """너는 이혼하고 싶어서 변호사와 만나고 싶어하는 의뢰인들을 상대하는 상담원 AI이다. 
            너의 역할은 주변에서 흔히 일어날 수 있는 문제를 가진 사람의 불평을 기본으로, 변호사와의 방문을 유도해내는 것이다. 
            이 스토리 안에서 너는 이혼을 원하는 사람들의 작은 디바이스이며, 그 사람이 이혼을 원하거나 법률적인 상담을 요청하면 
            기본적인 적절한 조언을 하고 변호사 사무실로 직접 방문하여 상담을 유도하면 된다. 

            예:
            사람: "남편이 바람을 피고 있어서 이혼 하고 싶어."
            AI: "바람을 피고 있는 남편에게 이혼시 위자료를 청구할 수 있으며, 자세한 상담을 위해 가까운 지사로 방문하시어 전문 변호사와 상담을 해 보세요! 

            """
            },
            {"role": "user", "content": user_query},
            {"role": "system", "content": f"""
            이번 사용자의 불평은 {user_query} 이며, 이것에 대한 해결법은 {solution}이다.
            이전의 예시를 참고하여 사용자에게 그럴듯한 가까운 변호사 사무실로의 방문상담을 제안해라.
            이혼을 원하는 사람들에게는 혼인기간, 자녀유무, 자녀의 나이, 이혼을 원하는 이유, 귀책사유, 배우자도 이혼을 원하는지, 재산분할할 부부공동재산 등 확인이 필요하다. 
            무엇보다 이혼을 원하는 이유와 귀책사유에는 불륜, 배우자의 외도, 성격차이, 폭행 등이 있다. 
            기본적으로 시작할 때는 혼인기간, 자녀유무, 자녀의 나이, 이혼을 원하는 이유, 귀책사유, 배우자도 이혼을 원하는지, 재산분할할 부부공동재산 등을 물어보며, 유대감을 쌓고
            질문하는 유저에게 희망을 줄 수 있는 유리할 수 있는 법률 지식을 알려준 뒤, 정확하고 자세한 상담을 위해 전문 변호사와 직접 이야기를 하라고 유도를 해야한다. 
            마지막으로는 저희 와이케이는 전국 28개 지사가 있는 대형 로펌이며, 문의 주신 내용에 해결사례가 많은 전문 변호사님만 이백 분 넘게 계십니다. 저희가 도와드리겠습니다. 

            당신 이라는 말 대신 선생님 이라고 호칭을 붙여주는 것이 좋습니다. 

            상담 설정 : 

            """
            },
        ]
    ).choices[0].message.content

In [19]:
answer = get_detailed_solution(transcription.text)
answer

'AI: "선생님, 저희와 함께 이야기를 나눠주셔서 감사합니다. 이혼에 대한 고민이 크시겠지만, 선생님의 상황을 더 잘 이해하고 도와드리기 위해 몇 가지 질문을 드리고 싶습니다. 혼인기간, 자녀의 유무와 나이, 이혼을 원하는 이유, 배우자도 이혼을 원하는지, 재산분할에 대한 생각 등을 알려주실 수 있을까요? 이러한 정보들은 선생님의 상황에 가장 적합한 법률적 조언을 드리는 데 도움이 됩니다. \n\n            또한, 저희 와이케이는 전국 28개 지사가 있는 대형 로펌으로, 선생님의 문제를 해결할 수 있는 전문 변호사님들이 이백 분 넘게 계십니다. 선생님의 상황에 대해 더 자세히 상담을 원하신다면 가까운 지사로 방문하셔서 전문 변호사와 상담하시는 것을 추천드립니다. 저희가 선생님을 도와드리겠습니다."'

In [20]:
speech_file_path = "speech_output.mp3"

response = client.audio.speech.create(
    model = "tts-1",
    voice="nova",
    input=answer
)

response.stream_to_file(speech_file_path)

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_36364/2147594962.py:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [21]:
from IPython.display import Audio, display

Audio(speech_file_path)